# Incrementando os Dados

Neste momento realizando um aumento dos dados que já temos disponíveis. Com uma análise na base de dados é possível criar novos dados, baseados nos dados já existentes, para poder criar novos insights e gerar conhecimento.

Ao decorrer do script iremos explicar o que cada passo está fazendo.

In [1]:
# Importação das libs necessárias para o script
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
import boto
import boto3
from io import StringIO
import warnings

# Removendo limite de colunas no Jupyter
pd.options.display.max_columns = None
warnings.filterwarnings('ignore')

# Leitura dos dados

In [2]:
# Definindo um parser para a Data existente no nosso dataset inicial
dateparse = lambda x: pd.datetime.strptime(x, '%d/%m/%Y %H:%M:%S')

# Lendo informações direto da AWS
# Dados da prefeitura de curitiba
df = pd.read_csv('s3://g13batalha/dataset_curitiba.csv',
                 sep=';', error_bad_lines=False, low_memory=False, encoding='windows_1252', 
                 parse_dates=['Data do Atendimento', 'Data de Nascimento'], date_parser=dateparse)

In [3]:
# Verificando os tipos de dados
df.dtypes

Data do Atendimento                             datetime64[ns]
Data de Nascimento                              datetime64[ns]
Sexo                                                    object
Código do Tipo de Unidade                                int64
Tipo de Unidade                                         object
Código da Unidade                                        int64
Descrição da Unidade                                    object
Código do Procedimento                                   int64
Descrição do Procedimento                               object
Código do CBO                                           object
Descrição do CBO                                        object
Código do CID                                           object
Descrição do CID                                        object
Solicitação de Exames                                   object
Qtde Prescrita Farmácia Curitibana                      object
Qtde Dispensada Farmácia Curitibana                    

# Preparação dos dados

In [4]:
# Pegando o dia do Atendimento
df['Dia do Atendimento'] = df['Data do Atendimento'].dt.weekday

# Criando uma métrica com somente a data do atendimento, será usada para cruzamento metereológico
df["so_data"] = pd.to_datetime(df['Data do Atendimento'].dt.date)

# Mapeamento de dia da semana para nomes em Portugues
# OBS: Ambiente da Amazon está configurado em US, não sendo possível alterar o locale do Python para pt_BR
dias = {0:'Segunda-feira', 
        1:'Terça-feira', 
        2:'Quarta-feira', 
        3:'Quinta-feira', 
        4:'Sexta-feira',
        5:'Sábado',
        6:'Domingo'}

# Realiando o map do dicionário acima
df['Dia do Atendimento'] = df['Dia do Atendimento'].map(dias)

# Criando métrica de hora de atendimento
df['Hora do Atendimento'] = df['Data do Atendimento'].dt.time

# Calculando a idade do paciente
df["Idade"] = df.apply(lambda x: relativedelta(x['Data do Atendimento'], x['Data de Nascimento']).years, axis=1)

In [5]:
# Adicionando base externa com endereços e geolocalização das bases de atendimento de Curitiba
df2 = pd.read_csv('s3://g13batalha/UPAS_Curitiba.csv',
                 sep=',', error_bad_lines=False, low_memory=False, encoding='utf-8')

In [6]:
# Join da base original com as informações geográficas
df = df.merge(df2, left_on="Descrição da Unidade", right_on="ÚNICO", how='left')
    
# Removendo colunas não mais necessárias
df.drop("ÚNICO", axis=1, inplace=True)

In [7]:
# Carregando informações técnicas dos Códigos de Identificação de Doenças
df3 = pd.read_csv('s3://g13batalha/CIDs.csv',
                 sep=',', error_bad_lines=False, low_memory=False, encoding='utf-8')

In [8]:
# Removimendo colunas sem informação
df3.dropna(axis=1, inplace=True)

# Renomeando as colunas que restaram
df3.columns = ['SUBCAT', 'Descrição Técnica do CID', 'Descrição CID Abreviada']

In [9]:
# Realizando Join das informações de CID com a base original da prefeitura
df = df.merge(df3, left_on="Código do CID", right_on="SUBCAT", how='left')

# Removendo colunas não mais necessárias
df.drop("SUBCAT", axis=1, inplace=True)

In [10]:
# Definindo padrão de data
dateparse = lambda x: pd.datetime.strptime(x, '%d/%m/%Y')

# Carregando dados externos de temperatura
df4 = pd.read_csv('s3://g13batalha/temp_JUL_NOV_2018.csv',
                 sep=',', error_bad_lines=False, low_memory=False, encoding='utf-8',
                 parse_dates=['data'], date_parser=dateparse)

In [11]:
# Alterando tipo de dado para cruzamento com o dataset original
df4["data"] = pd.to_datetime(df4.data)

# Criando métrica de temperatura média
df4["media"] = (df4["max"]+df4["min"])/2

# Renomeando as colunas
df4.columns = ['data', 'temp_max', 'temp_min', 'temp_media']

In [12]:
# Realizando join das temperaturas com a base original, pela data
df = df.merge(df4, left_on="so_data", right_on="data", how='left')

# Removendo colunas não mais necessárias
df.drop("so_data", axis=1, inplace=True)
df.drop("data", axis=1, inplace=True)

In [13]:
# Ordenando os campos
lista_ordenada = ['Data do Atendimento', 'Dia do Atendimento', 'Hora do Atendimento', 'Data de Nascimento', 
                  'Idade', 'Sexo', 'Código do Tipo de Unidade', 'Tipo de Unidade', 'Código da Unidade', 
                  'Descrição da Unidade', 'Municício', 'Bairro', 'Endereco', 'Lat', 'Long', 'Código do Procedimento',
                  'Descrição do Procedimento', 'Código do CBO', 'Descrição do CBO', 'Código do CID', 'Descrição do CID', 
                  'Descrição Técnica do CID', 'Descrição CID Abreviada', 'Solicitação de Exames', 'Qtde Prescrita Farmácia Curitibana',
                  'Qtde Dispensada Farmácia Curitibana', 'Qtde de Medicamento Não Padronizado', 'Encaminhamento para Atendimento Especialista', 
                  'Área de Atuação', 'Desencadeou Internamento', 'Data do Internamento', 'Estabelecimento Solicitante', 'Estabelecimento Destino',
                  'CID do Internamento', 'Tratamento no Domicílio', 'Abastecimento', 'Energia Elétrica', 'Tipo de Habitação', 'Destino Lixo', 
                  'Fezes/Urina', 'Cômodos', 'Em Caso de Doença', 'Grupo Comunitário', 'Meio de Comunicacao', 'Meio de Transporte', 'temp_max', 'temp_min', 'temp_media']

# Guardando as informações ordenadas
df = df[lista_ordenada]

In [14]:
# Visualização do dataset final
df.head(2)

,Data do Atendimento,Dia do Atendimento,Hora do Atendimento,Data de Nascimento,Idade,Sexo,Código do Tipo de Unidade,Tipo de Unidade,Código da Unidade,Descrição da Unidade,Municício,Bairro,Endereco,Lat,Long,Código do Procedimento,Descrição do Procedimento,Código do CBO,Descrição do CBO,Código do CID,Descrição do CID,Descrição Técnica do CID,Descrição CID Abreviada,Solicitação de Exames,Qtde Prescrita Farmácia Curitibana,Qtde Dispensada Farmácia Curitibana,Qtde de Medicamento Não Padronizado,Encaminhamento para Atendimento Especialista,Área de Atuação,Desencadeou Internamento,Data do Internamento,Estabelecimento Solicitante,Estabelecimento Destino,CID do Internamento,Tratamento no Domicílio,Abastecimento,Energia Elétrica,Tipo de Habitação,Destino Lixo,Fezes/Urina,Cômodos,Em Caso de Doença,Grupo Comunitário,Meio de Comunicacao,Meio de Transporte,temp_max,temp_min,temp_media
0,2018-09-18 20:39:46,Terça-feira,20:39:46,2011-03-12,7,F,2,UPA,17396,UPA BOA VISTA,PORTO AMAZONAS,BAIRRO NAO INFORMADO ...,"Av. Parana 3654, Curitiba, PR, 82510-000",-25.385328,-49.232841,301060096,ATENDIMENTO MEDICO EM UNIDADE DE PRONTO ATENDI...,225125,MEDICO CLINICO,J069,INFECCAO AGUDA DAS VIAS AEREAS SUPERIORES NAO ...,Infecção aguda das vias aéreas superiores não ...,J06.9 Infecc aguda das vias aereas super NE,Nao,2,0,0,Nao,NaN,Nao,NaN,NaN,NaN,NaN,NaN,NaN,Sim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24,13,18.5
1,2018-09-30 18:38:45,Domingo,18:38:45,1984-09-23,34,F,2,UPA,5323495,UPA CIDADE INDUSTRIAL,CURITIBA,CIDADE INDUSTRIAL,"Rua Sen. Accioly Filho 3370, Curitiba, PR, 813...",-25.497540,-49.339230,301060029,ATENDIMENTO DE URGENCIA C/ OBSERVACAO ATE 24 H...,225125,MEDICO CLINICO,N390,INFECCAO DO TRATO URINARIO DE LOCALIZACAO NAO ...,NaN,NaN,Nao,49,0,0,Nao,NaN,Nao,NaN,NaN,NaN,NaN,NaN,NaN,Sim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23,16,19.5


# Exportação dos Dados

In [15]:
# Exportando o dataset para csv, direto para o bucket do S3.
csv_buffer = StringIO()
df.to_csv(csv_buffer, encoding='windows_1252')
s3_resource = boto3.resource('s3')
s3_resource.Object('g13batalha', 'dataset_final.csv').put(Body=csv_buffer.getvalue())

#df.to_csv('temp.csv', sep=';')

{'ETag': '"9c57729bfb23590360801c7566377dbb"',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Sun, 09 Dec 2018 09:42:07 GMT',
   'etag': '"9c57729bfb23590360801c7566377dbb"',
   'server': 'AmazonS3',
   'x-amz-id-2': 'OLyqej+c+W02IOaEzjFwQzPCvhU/XB5neKqbtOFwH3KU/fR1QPJvqY8yJe9pmuYBn9OhDTMmEuY=',
   'x-amz-request-id': 'ACC066814D824B6B'},
  'HTTPStatusCode': 200,
  'HostId': 'OLyqej+c+W02IOaEzjFwQzPCvhU/XB5neKqbtOFwH3KU/fR1QPJvqY8yJe9pmuYBn9OhDTMmEuY=',
  'RequestId': 'ACC066814D824B6B',
  'RetryAttempts': 0}}

# Referencias:

## Dicionário de dados final

Relação das coluna e seu significado:

| Campo | Descrição | Tipo | Origem |
|---|---|---|
| Data do Atendimento | Data de Realização do Atendimento | Data | Prefeitura de Curitiba |
| Dia do Atendimento | Nome do dia da Semana do Atendimento | Texto | Composto do campo 'Data do Atendimento' |
| Hora do Atendimento | Horário do Atendimento | Hora | Composto do campo 'Data do Atendimento' | 
| Data de Nascimento | Data de Nascimento do Paciente | Data | Prefeitura de Curitiba |
| Idade | Idade do Paciente | Numérico | Calculado: 'Data do Atendimento' - 'Data de Nascimento' |
| Sexo | Sexo do Paciente | Texto | Prefeitura de Curitiba |
| Código do Tipo de Unidade | Código do Tipo de Unidade de Atendimento | Numérico | Prefeitura de Curitiba |
| Tipo de Unidade | Tipo de Unidade de Atendimento | Texto | Prefeitura de Curitiba |
| Código da Unidade | Código da Unidade de Atendimento | Texto | Prefeitura de Curitiba |
| Descrição da Unidade | Descrição da Unidade de Atendimento | Texto | Prefeitura de Curitiba |
| Municício | Município do paciente | Texto | Prefeitura de Curitiba |
| Bairro | Bairro do paciente | Texto | Prefeitura de Curitiba |
| Endereco | Nacionalidade do paciente (Brasileira, Naturalizado, Estrangeiro e Não informado) | Texto | Prefeitura de Curitiba |
| Lat | Latitude da Unidade de Atendimento | Numérico | Obtido através do Google Maps |
| Long | Longitude da Unidade de Atendimento | Numérico | Obtido através do Google Maps
| Código do Procedimento | Código do Procedimento Realizado | Texto | Prefeitura de Curitiba |
| Descrição do Procedimento | Descrição do Procedimento Realizado | Texto | Prefeitura de Curitiba |
| Código do CBO | Código da Ocupação do Profissional | Texto | Prefeitura de Curitiba |
| Descrição do CBO | Descrição da Ocupação do Profissional | Texto | Prefeitura de Curitiba |
| Código do CID | Código do Diagnóstico | Texto | Prefeitura de Curitiba |
| Descrição do CID | Descrição do Diagnóstico | Texto | Prefeitura de Curitiba |
| Descrição Técnica do CID | Descrição fornecida pelo Código único de Doenças | Texto | Datasus.gov |
| Descrição CID Abreviada | Descrição Abreviada do Código único de Doenças | Texto | Datasus.gov |
| Solicitação de Exames | Indica se ocorreu solicitação de Exames | Texto | Prefeitura de Curitiba |
| Qtde Prescrita Farmácia Curitibana | Qtde de medicamentos prescritos na Farmácia Curitibana | Numérico | Prefeitura de Curitiba |
| Qtde Dispensada Farmácia Curitibana | Qtde de medicamentos dispensados na Farmácia Curitibana | Numérico | Prefeitura de Curitiba |
| Qtde de Medicamento Não Padronizado | Qtde de Medicamento Não Padronizado | Numérico | Prefeitura de Curitiba |
| Encaminhamento para Atendimento Especialista | Indica se houve encaminhamento para Atendimento de Especialista |  | Prefeitura de Curitiba |
| Área de Atuação | Área de Atuação | Texto | Prefeitura de Curitiba |
| Desencadeou Internamento | Indica se desencadeou Internamento | Texto | Prefeitura de Curitiba |
| Data do Internamento | Data do Internamento do paciente | Data | Prefeitura de Curitiba |
| Estabelecimento Solicitante | Estabelecimento que solicitou o internamento | Texto | Prefeitura de Curitiba |
| Estabelecimento Destino | Estabelecimento que houve a internação | Texto | Prefeitura de Curitiba |
| CID do Internamento | Código do diagnóstico do internamento | Texto | Prefeitura de Curitiba |
| Tratamento no Domicílio | Tipo de Tratamento de Água no domicílio | Texto | Prefeitura de Curitiba |
| Abastecimento | Tipo de Abastecimento de Água no domicílio | Texto | Prefeitura de Curitiba |
| Energia Elétrica | Indica se há energia elétrica no domicílio | Texto | Prefeitura de Curitiba |
| Tipo de Habitação | Tipo de habitação no domicílio | Texto | Prefeitura de Curitiba |
| Destino Lixo | Destino do lixo no domicílio | Texto | Prefeitura de Curitiba |
| Fezes/Urina | Destino das fezes/urina no domicílio | Texto | Prefeitura de Curitiba |
| Cômodos | Qtde de Cômodos no domicílio | Numérico | Prefeitura de Curitiba |
| Em Caso de Doença | Serviços procurados em caso de doença | Texto | Prefeitura de Curitiba |
| Grupo Comunitário | Grupo Comunitário em que o paciente participa | Texto | Prefeitura de Curitiba |
| Meio de Comunicacao | Meios de Comunicação utilizados no domicílio | Texto | Prefeitura de Curitiba |
| Meio de Transporte | Meios de Transporte utilizados no domicílio | Texto | Prefeitura de Curitiba |
| Temp Max | Temperatura Máxima prevista para o dia de atendimento | Decimal | Accuweather |
| Temp Min | Temperatura Mínima prevista para o dia de atendimento | Decimal | Accuweather |
| Temp Media | Temperatura Média prevista para o dia de atendimento | Decimal | Accuweather |

## Origem dos dados:
Prefeitura de Curitiba: http://www.curitiba.pr.gov.br/dadosabertos/consulta/?grupo=1  

Arquivos:  
Sistema E-Saude - Perfil de atendimento nas Unidades Municipais de Saúde de Curitiba;  
Sistema E-Saude - Perfil de atendimento de Enfermagem nas Unidades Municipais de Saúde de Curitiba

Informações do CID: http://www.datasus.gov.br/cid10/V2008/cid10.htm  
             
Informações de Geolocalização: https://www.google.com.br/maps

Informações Metereológicas: https://www.accuweather.com/pt/br/curitiba/44944/weather-forecast/44944